In [ ]:
#### 네이버 기사 제목 데이터 크롤링
- selenium을 이용한 텍스트 데이터 수집
- thread를 사용한 크롤링
    - idle타임에 또다른 리퀘스트를 하며 클라이언트 서버가 놀지 않는다 => 시간단축
    - 이렇게 컴퓨팅 파워를 관리하는것에 관심이 있다면 celery를 공부해라
- scrapy를 이용하면 다음과 같이 설정하지 않아도 자동으로 thread를 이용해 크롤링해준다

In [1]:
from selenium import webdriver

In [3]:
page = 2

url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105#&date=%2000:00:00&page={}'.format(page)

In [4]:
driver =  webdriver.Chrome()
driver.get(url)

In [5]:
#여러데이터 모으기
articles = driver.find_elements_by_css_selector("#section_body li")
len(articles)

20

In [6]:
articles[0].find_element_by_css_selector("dt:not(.photo) > a").text

"판은 한국이 깔았는데…'중국 놀이터' 된 지스타 2019 [노정동의 3분IT]"

In [7]:
article_list = []
for article in articles:
    title = article.find_element_by_css_selector("dt:not(.photo) > a").text
    article_list.append(title)

In [8]:
article_list

["판은 한국이 깔았는데…'중국 놀이터' 된 지스타 2019 [노정동의 3분IT]",
 '무선 게임패드 ‘샥스’, 론칭 2달만에 가성비로 주목',
 "'카트라이더: 드리프트', 런던서 최초 공개",
 "[지스타]넷마블·펄어비스 신작부스 '북적'…참여형 e스포츠 '인기'",
 "'韓게임, 판매는 막고 내용은 베끼고'…中게임 국내 침투 가속",
 '정부 "소형원전 \'스마트\' 상용화 의지 밝혀" 탈원전 정책은 내수용?',
 "문미옥 차관, 美서 소형원자로 '스마트' 우수성·수출의지 알려",
 '신학철 LG화학 "배터리 산업, \'소·부·장\'에 달렸다"',
 '정부 AI국 신설에 "컨트롤타워 환영...산업계와 긴밀히 소통해야"',
 "中 화웨이·모토로라도 폴더블폰…韓 삼성의 '한 수 아래'",
 '"포항지진 1년전 물 주입 중단했다면 지진 확률 1%로 줄었을 것"',
 "'갤폴드' 완판에 다급한 화웨이…품질논란 '메이트X' 출격",
 '"우주전파재난 대비 독자 위성관측 로드맵 만든다"',
 "삼성전자, 日서 스마트폰 시장 점유율 '상승곡선' 반등 '청신호'",
 'MS-서울대, 인공지능 공동 연구 나섰다',
 '[지스타 2019] 전 세계 게임이 모바일 중심으로 재편... LG전자, 듀얼 스크린 존재감 과시',
 'LG화학 신학철 대표, “배터리 제패, 소재·부품·장비에 달렸다”',
 '소형 원자로 개발 주도국 한자리에',
 '위메이드, ‘미르의전설2’ IP 한 데 담은 플랫폼 구상',
 "과기정통부, 초대 AI국장에 강도현 배치...'AI 국가전략' 구심체(종합)"]

In [9]:
driver.quit()

In [10]:
article_list = []
def get_articles(page):
    driver =  webdriver.Chrome()
    url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105#&date=%2000:00:00&page={}'.format(page)
    driver.get(url)
    #여러데이터 모으기
    articles = driver.find_elements_by_css_selector("#section_body li")
    for article in articles:
        title = article.find_element_by_css_selector("dt:not(.photo) > a").text
        article_list.append(title)
    print("crawling : {}".format(page))
    driver.quit()

In [11]:
%%time
for page in range(1,5):
    get_articles(page)

crawling : 1
crawling : 2
crawling : 3
crawling : 4
Wall time: 38.7 s


In [13]:
article_list[:3]

["5년 뒤 9兆 시장…'게임체인저' 주목받는 클라우드게임",
 '"소규모 지진 때 지열발전 멈췄다면…포항지진 확률 1%에 불과했을 것"',
 '"영하 5도 이하에선 접지 말라"…그래도 1분만에 \'매진\'된 메이트X']

#### thread 사용

In [16]:
import threading

In [19]:
article_list = []

for page in range(1,5):
    th = threading.Thread(target=get_articles, args=(page,))  #실행시킬 함수의 이름, (page,)=>데이터가 하나있는 튜플
    th.start()

In [20]:
import time
sec = 0
while len(article_list)< 80:
    time.sleep(1)
    sec +=1     #1초마다 sec이 1씩 증가
    print(sec, end=" ")

1 2 3 4 5 6 7 8 9 10 11 12 crawling : 3
13 crawling : 4
crawling : 2
crawling : 1
14 